In [ ]:
# %pip install ipywidgets
# %pip install jupyter
# %pip install accelerate -U

In [1]:
from tqdm.notebook import tqdm

from datasets import load_dataset
from nltk.tokenize import sent_tokenize
import torch
from transformers import BertForMaskedLM, BertTokenizer, logging

from blanc import BLANC_tune, add_results_to_json

[nltk_data] Downloading package punkt to /Users/nazanin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
logging.set_verbosity_warning()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_checkpoint = "bert-base-uncased"
bert_model = BertForMaskedLM.from_pretrained(bert_checkpoint).to(DEVICE)
bert_tokenizer = BertTokenizer.from_pretrained(bert_checkpoint, do_lower_case=True)

mbert_checkpoint = "bert-base-multilingual-uncased"
mbert_model = BertForMaskedLM.from_pretrained(mbert_checkpoint).to(DEVICE)
mbert_tokenizer = BertTokenizer.from_pretrained(mbert_checkpoint, do_lower_case=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint

# BLANC tune for **summaries**

In [3]:
""" Datasets """

DailyNews_ds = load_dataset("json", data_files="../datasets/DailyNews_300.json", split="train")
DailyNews_ds

Dataset({
    features: ['summary', 'text', 'annotators_ids', 'scores'],
    num_rows: 300
})

In [4]:
""" Preprocessing """

summaries = DailyNews_ds["summary"]  # (List[str])
texts = DailyNews_ds[
    "text"
]  # (List[str]) each string is a paragraph made of a few sentences

# each text in texts is a list of sentences (each sentence is a string)
texts = [sent_tokenize(text.strip()) for text in texts]  # List[List[str]]
assert len(texts) == len(summaries) == 300

tokenized_texts = [
    [bert_tokenizer.tokenize(sentence) for sentence in text] for text in texts
]  # List[List[List[str]]]
tokenized_summaries = [
    bert_tokenizer.tokenize(summary) for summary in summaries
]  # [List[List[str]]]

In [5]:
""" Running the Program """

tune_summary_scores = [
    BLANC_tune(
        text, summary, bert_checkpoint, bert_model, bert_tokenizer, device=DEVICE
    )
    for text, summary in tqdm(
        zip(tokenized_texts, tokenized_summaries), total=len(tokenized_texts)
    )
]

# Saving the results
# summary_data = {}
# summary_data["BLANC_tune_summary"] = tune_summary_scores
# add_results_to_json(summary_data)

  0%|          | 0/300 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/21 [00:00<?, ?it/s]

{'train_runtime': 24.1766, 'train_samples_per_second': 6.204, 'train_steps_per_second': 0.869, 'train_loss': 0.19191932678222656, 'epoch': 3.0}


RuntimeError: Placeholder storage has not been allocated on MPS device!

# BLANC tune for **translations**

In [3]:
""" Datasets """

# English - French
# en_fr_df = pd.read_csv("../datasets/en_to_fr_translations.csv", index_col=0)
en_fr_ds = load_dataset("csv", data_files="../datasets/en_to_fr_translations.csv", split="train", index_col=0)

# English - Persian (Farsi)
en_fa_ds = load_dataset("persiannlp/parsinlu_translation_en_fa", split="train")

In [4]:
""" Preprocessing (English - French)"""

# Tokenization
en_fr_sentences = [
    mbert_tokenizer.tokenize(sentence) for sentence in en_fr_ds["en"]
]  # (List[List[str]])

en_fr_translations = [
    mbert_tokenizer.tokenize(translation) for translation in en_fr_ds["fr"]
]  # (List[List[str]])


""" Preprocessing (English - Persian (Farsi)) """

# Removing the 'category' column
en_fa_ds = en_fa_ds.remove_columns(["category"])

# Removing list encapsulation
en_fa_ds = en_fa_ds.map(lambda example: {"targets": example["targets"][0]}, num_proc=4)

# Filtering out:
# - rows with the '\u200c' symbol,
# - those where the length of either source or targets is less than a threshold
# - Headlines (ending in 'Global Voices') --> because they are very short and the 'Global Voices' part is never translated
length_threshold = 30
filtered_en_fa_ds = en_fa_ds.filter(
    lambda example: "\u200c" not in example["targets"]
    and len(example["source"]) >= 30
    and len(example["targets"]) >= 30
    and "Global Voices" not in example["source"],
    num_proc=4,
)

en_fa_ds = (
    filtered_en_fa_ds.rename_column("source", "sentence")
    .rename_column("targets", "translation")
    .select(range(300))
)

# Tokenization
en_fa_sentences = [
    mbert_tokenizer.tokenize(sentence) for sentence in en_fa_ds["sentence"]
]  # (List[List[str]])

en_fa_translations = [
    mbert_tokenizer.tokenize(translation) for translation in en_fa_ds["translation"]
]  # (List[List[str]])

In [ ]:
""" Running the Program (English - French)"""

tune_en_fr_scores = [
    BLANC_tune(
        sentences,
        translations,
        mbert_checkpoint,
        mbert_model,
        mbert_tokenizer,
        device=DEVICE,
        is_translation=True
    )
    for sentences, translations in tqdm(
        zip(en_fr_sentences, en_fr_translations), total=len(en_fr_sentences)
    )
]

# Saving the results
# en_fr_data = {}
# en_fr_data["BLANC_tune_en_fa_translation"] = tune_en_fr_scores
# add_results_to_json(en_fr_data)

In [ ]:
""" Running the Program (English - Persian)"""

tune_en_fa_scores = [
    BLANC_tune(
        sentences,
        translations,
        mbert_checkpoint,
        mbert_model,
        mbert_tokenizer,
        device=DEVICE,
        is_translation=True
    )
    for sentences, translations in tqdm(
        zip(en_fa_sentences, en_fa_translations), total=len(en_fa_sentences)
    )
]

# Saving the results
# en_fa_data = {}
# en_fa_data["BLANC_tune_en_fa_translation"] = tune_en_fa_scores
# add_results_to_json(en_fa_data)